## 1. Installation

In [ ]:
!git clone https://github.com/cantaro86/Financial-Models-Numerical-Methods
%cd Financial-Models-Numerical-Methods
!pip install --requirement list_of_packages.txt
!pip install -e .

fatal: destination path 'Financial-Models-Numerical-Methods' already exists and is not an empty directory.
/content/Financial-Models-Numerical-Methods
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.1/985.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
  

## 2. Black-Scholes Pricer

### 2.1 Put-Call parity test function

In [ ]:
def put_call_parity(call_price, put_price):
  put = put_price + S0 - K * np.exp(-r * T)
  check = call_price == put
  print(f"CHECK: Put-Call Parity: \n Put Price: {put} \n Call Price: {call_price} \n CHECK: {check}")


In [ ]:
from FMNM.BS_pricer import BS_pricer

def bs_pricer(S0=100, K=100, T=1, r=0.1, sig=0.2):

  call = BS_pricer.BlackScholes("call", S0, K, T, r, sig)
  put = BS_pricer.BlackScholes("put", S0, K, T, r, sig)
  print("Call price: ", call)
  print("Put price: ", put)

  return [call, put]

soln = bs_pricer()
put_call_parity(soln[0], soln[1])

## 3. Option Pricer with other models

In [ ]:
from FMNM.Parameters import Option_param
from FMNM.Processes import Diffusion_process

def pricer(S0=100, K=100, T=1, r=0.1, sig=0.2):
  # Exercise = American | European
  opt_param = Option_param(S0, K, T, exercise="European", payoff="call")
  diff_param = Diffusion_process(r, sig)
  # Creates the pricer object
  BS = BS_pricer(opt_param, diff_param)
  BS.closed_formula()
  13.269676584660893
  BS.Fourier_inversion()
  13.269676584660623
  BS.MC(N=30000000, Err=True, Time=True)
  # output is: price, standard error and execution time
  # (array([13.26753511]), array([0.00294085]), 0.679786205291748)
  BS.mesh_plt()  # PDE method

import numpy as np
import scipy.stats as ss

def monte_carlo_pricer(S0=100, K=100, T=1, r=0.1, sig=0.2):
  np.random.seed(seed=44)  # seed for random number generation
  N = 10000000  # Number of random variables

  W = ss.norm.rvs((r - 0.5 * sig**2) * T, np.sqrt(T) * sig, N)
  S_T = S0 * np.exp(W)

  call = np.mean(np.exp(-r * T) * np.maximum(S_T - K, 0))
  put = np.mean(np.exp(-r * T) * np.maximum(K - S_T, 0))
  call_err = ss.sem(np.exp(-r * T) * np.maximum(S_T - K, 0))  # standard error
  put_err = ss.sem(np.exp(-r * T) * np.maximum(K - S_T, 0))  # standard error

from functools import partial
import matplotlib.pyplot as plt
%matplotlib inline

def pricer_model_limits(S0=100, K=100, T=1, r=0.1, sig=0.2):
  BS_sigma = partial(BS_pricer.BlackScholes, "call", S0, K, T, r)  # binding the function
  sigmas = np.linspace(0.01, 10, 1000)

  plt.plot(sigmas, BS_sigma(sigmas))
  plt.xlabel("sig")
  plt.ylabel("price")
  plt.title("Black-Scholes price as function of volatility")
  plt.show()

pricer()
monte_carlo_pricer()
pricer_model_limits()